In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import torch
import torch_geometric
import torch
import os
import os.path as osp
import numpy as np
from tqdm import tqdm as tqdm
torch.manual_seed(1)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from torch_geometric.data import Data

class ClusData(Data):
    def __init__(self, edge_index_clus=None, x_clus=None, edge_index_nodex=None, x_nodex=None):
        super().__init__()
        self.edge_index_clus = edge_index_clus
        self.x_clus = x_clus
        self.edge_index_nodex = edge_index_nodex
        self.x_nodex = x_nodex
        
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index_clus':
            return self.x_clus.size(0)
        if key == 'edge_index_nodex':
            return self.x_nodex.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)


In [ ]:
# import electron datasets
import glob
raw_dir="/grid_mnt/data__data.polcms/cms/sghosh/hackathon/graphsv2_thr0p2phi0p2_clusprops/"
#raw_dir='/data_CMS/cms/sghosh/elecgraphs/V1condor/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data*.pt')]
data_list_ele = []
etav = []
ctr = 0
for i in tqdm(fnamelist):
    idx = torch.load(i)
    #print(idx.x.shape[0])
    #idx.x = idx.clusx
    '''print(idx.keys)
    break'''
    
    if idx.nodex.shape[0] < 3:
        continue
    if not(np.any(idx.nodelabel.numpy())):  ### remove where there are no eles matched
        continue
        
    data = ClusData(x_clus = idx.clusx, x_nodex = idx.nodex,edge_index_nodex=idx.edgeidx.long())    
    data.edgelabel = idx.edgelabel
    data.clusidx = idx.clusidx
    #idx.edge_index = idx.edgeidx.long()
    #idx.edgeidx = []
    #idx.clusidx = torch.tensor(torch.unsqueeze(idx.clusidx,dim=1),dtype=torch.int32)
    data_list_ele.append(data)
    #ctr += 1
    #if ctr > 200:
    #    break
    #data_list.append(torch.load(i))
    

#data_list=[x for x in data_list if (x.y<100.)]   
print((data_list_ele[0].x))
#print(data_list_pho[0].z)
print(len(data_list_ele))
totalevele = len(data_list_ele)
#trainev =

In [ ]:
data_list_ele[0].clusidx

In [ ]:
#del data_list_comb
totalevele = len(data_list_ele)
ntrain = int(0.8*totalevele)
import random


#random.shuffle(data_list_ele)
import torch_geometric
ntrainbatch = 100
ntestbatch = 100
trainloader = torch_geometric.data.DataLoader(data_list_ele[:ntrain], batch_size=ntrainbatch,follow_batch=['x_clus'])
testloader = torch_geometric.data.DataLoader(data_list_ele[ntrain:], batch_size=ntestbatch,follow_batch=['x_clus'])
#batch_size = ntrainbatch
epoch_size = len(data_list_ele[:ntrain])
print("epoch size,batch_size:",epoch_size,ntrainbatch)

In [ ]:
for data in trainloader:
    print(data)
    print(len(data.x_nodex))
    break

In [ ]:
import os
import os.path as osp
import math

import numpy as np
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch.utils.checkpoint import checkpoint
from torch_cluster import knn_graph

from torch_geometric.nn import EdgeConv, NNConv
#from torch_geometric.nn.pool.edge_pool import EdgePooling

from torch_geometric.utils import normalized_cut
from torch_geometric.utils import remove_self_loops
from torch_geometric.utils.undirected import to_undirected
from torch_geometric.nn import (graclus, max_pool, max_pool_x,
                                global_mean_pool, global_max_pool,
                                global_add_pool,BatchNorm)

transform = T.Cartesian(cat=False)

def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

class ClusNetwork(nn.Module):
    # Nodeclassification network 
    
    def __init__(self, 
                 inputclus_dim=4,
                 hiddenclus_dim=20,
                 input_dim=64,
                 hidden_dim=64,
                 output_dim=1,
                 clusk = 16,
                 k=16, 
                 aggr='add',
                 clusnorm=torch.tensor([1./500., 1./500., 1./54., 1/25., 1./1000.]),
                 norm=torch.tensor([1./500., 1./500., 1./54., 1/25., 1./1000.])):
 #                norm=torch.tensor([1., 1., 1., 1., 1.])):
        super(ClusNetwork, self).__init__()

        self.datanorm = nn.Parameter(norm,requires_grad=False)
        self.dataclusnorm = nn.Parameter(clusnorm,requires_grad=False)
        
        self.k = k
        self.clusk = clusk
        startclus_width = 2 * hiddenclus_dim
        middleclus_width = 3 * hiddenclus_dim // 2
        
        start_width = 2 * hidden_dim
        middle_width = 3 * hidden_dim // 2
        
        
        self.clusinputnet =  nn.Sequential(
            nn.Linear(inputclus_dim, hiddenclus_dim//2),            
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(hiddenclus_dim//2, hiddenclus_dim//2),
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(hiddenclus_dim//2, hiddenclus_dim),
            nn.ELU(),
        )
        self.inputnet =  nn.Sequential(
            nn.Linear(input_dim, hidden_dim//2),            
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim//2, hidden_dim//2),
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim//2, hidden_dim),
            nn.ELU(),
        )
        
        convnn1 = nn.Sequential(nn.Linear(startclus_width, middleclus_width),
                                nn.ELU(),
                                nn.Dropout(0.2),
                                nn.Linear(middleclus_width, hiddenclus_dim),                                             
                                nn.ELU()
                                )
        self.edgeconv1 = EdgeConv(nn=convnn1, aggr=aggr)
        
        convnn2 = nn.Sequential(nn.Linear(startclus_width, middleclus_width),
                                nn.ELU(),
                                nn.Dropout(0.2),
                                nn.Linear(middleclus_width, hiddenclus_dim),                                             
                                nn.ELU()
                                )
        self.edgeconv2 = EdgeConv(nn=convnn2, aggr=aggr)
        
        convnn3 = nn.Sequential(nn.Linear(startclus_width, middleclus_width),
                                nn.ELU(),
                                nn.Dropout(0.2),
                                nn.Linear(middleclus_width, hiddenclus_dim),                                             
                                nn.ELU()
                                )
        self.edgeconv3 = EdgeConv(nn=convnn3, aggr=aggr)
        
        
        convnn4 = nn.Sequential(nn.Linear(startclus_width, middleclus_width),
                                nn.ELU(),
                                nn.Dropout(0.2),
                                nn.Linear(middleclus_width, hiddenclus_dim),                                             
                                nn.ELU()
                                )
        self.edgeconv4 = EdgeConv(nn=convnn4, aggr=aggr)
        
        convnn5 = nn.Sequential(nn.Linear(startclus_width, middleclus_width),
                                nn.ELU(),
                                nn.Dropout(0.2),
                                nn.Linear(middleclus_width, hiddenclus_dim),                                             
                                nn.ELU()
                                )
        self.edgeconv5 = EdgeConv(nn=convnn5, aggr=aggr)
        
        
        convnn6 = nn.Sequential(nn.Linear(startclus_width, middleclus_width),
                                nn.ELU(),
                                nn.Dropout(0.2),
                                nn.Linear(middleclus_width, hiddenclus_dim),                                             
                                nn.ELU()
                                )
        self.edgeconv6 = EdgeConv(nn=convnn6, aggr=aggr)
        
        self.edge_classifier = nn.Sequential(
            nn.Linear(2*hiddenclus_dim, hiddenclus_dim),
            nn.ELU(),
            nn.BatchNorm1d(num_features=hiddenclus_dim),
            nn.Linear(hiddenclus_dim, hiddenclus_dim//2),
            nn.ELU(),
            nn.BatchNorm1d(num_features=hiddenclus_dim//2),
            nn.Linear(hiddenclus_dim//2, output_dim)
        )
        
        
                
        
        
        self.output = nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                    nn.ELU(),
                                    nn.Dropout(0.2),
                                    #nn.Softplus(),
                                    nn.Linear(hidden_dim, hidden_dim//2),
                                    nn.ELU(),
                                    nn.Dropout(0.2),
                                    #nn.Softplus(),
#                                    nn.Linear(hidden_dim//2, hidden_dim//2),#added
 #                                   nn.ELU(),
                                    #nn.Softplus(),
                                    nn.Linear(hidden_dim//2, output_dim)
                                   )
        self.batchnorm1 = BatchNorm(hidden_dim)
        
    def forward(self, data):
        #getting the batch in batch indices
        #print(data.x_clus_batch)
        orig_edge_idx = data.edge_index_nodex.clone()
        arrlist=[]
        for i,j in enumerate(data.clusidx):
            #print(i,j.numpy())
            arrlist.append(np.full(j.detach().cpu().numpy(),i))
        arrlist = np.concatenate(arrlist)
        #batchinbatch = torch.tensor(arrlist,dtype=torch.int64,device=device)
        batchinbatch = torch.tensor(arrlist,dtype=torch.int64).to(device) 
        #edgeconv for the clusters
        data.x_clus = self.clusinputnet(data.x_clus)
        edge_index = to_undirected(knn_graph(data.x_clus, self.clusk, batchinbatch, loop=False, flow=self.edgeconv1.flow))
        data.x_clus = self.edgeconv1(data.x_clus, edge_index)
        edge_index = to_undirected(knn_graph(data.x_clus, self.clusk, batchinbatch, loop=False, flow=self.edgeconv2.flow))
        data.x_clus = self.edgeconv2(data.x_clus, edge_index)
        edge_index = to_undirected(knn_graph(data.x_clus, self.clusk, batchinbatch, loop=False, flow=self.edgeconv3.flow))
        data.x_clus = self.edgeconv3(data.x_clus, edge_index)
        '''
        print("batchinbatch",batchinbatch)
        print("data.batch",data.batch)
        
        print("x_clus.shape",x_clus.shape)
        print("x_clus.batch",batch)'''
        
        x_clus, batch = max_pool_x(batchinbatch, data.x_clus, data.x_clus_batch)
        #print("x_clus.shape",x_clus.shape)
        #print("data.edge_index",data.edge_index)
        #print("data.edgelabel",data.edgelabel.shape)
        #print("data.nodex",data.nodex.shape)
        
        #data.edge_index = to_undirected(knn_graph(x_clus, self.k, batch, loop=False, flow=self.edgeconv4.flow))
        x_clus = self.edgeconv4(x_clus, orig_edge_idx)
        #data.edge_index = to_undirected(knn_graph(x_clus, self.k, batch, loop=False, flow=self.edgeconv5.flow))
        x_clus = self.edgeconv5(x_clus, orig_edge_idx)
        
        src, dst = orig_edge_idx
        edge_scores = self.edge_classifier(torch.cat([x_clus[src], 
                                                      x_clus[dst]], 
                                                      dim=-1)).squeeze(-1)
        
        '''edge_scores = torch.cat([x_clus[src], 
                                x_clus[dst]], 
                                dim=-1)'''
        
        
        
        return edge_scores
        
        
        
        
        '''src, dst = data.edge_index
        #score = (data.x[src] * data.x[dst]).sum(dim=-1)
        score = data.x[src] - data.x[dst]
        return score.squeeze(-1)
        
        return 0'''

In [ ]:
from lr_regulation import *

from tqdm import tqdm_notebook as tqdm


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drn = ClusNetwork(inputclus_dim=4,
                                           hiddenclus_dim=20, #50
                                           k=6,#16
                                           clusk = 16,
                                           output_dim=2,
                                           norm=torch.tensor([1., 1., 1., 1.,1.,1.,1.,1.]))
        
    def forward(self, data):
        logits = self.drn(data)
        #return F.softplus(logits)
        #return torch.sigmoid(logits)
        return logits
device = torch.device('cuda:1')#('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
scheduler = CyclicLRWithRestarts(optimizer, ntrainbatch, epoch_size, restart_period=200, t_mult=1.2, policy="cosine")

from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum() 

focalloss = FocalLoss()        


     
#losscat = torch.nn.CrossEntropyLoss(weight=torch.tensor([1., 2.],device="cuda:1"))    
losscat = torch.nn.CrossEntropyLoss(weight=torch.tensor([1., 50.],device="cuda:1"))    
def categorical_loss(outputa,trutha,alpha):
    total_loss =  losscat(outputa[:,:2],trutha[:,0].long()) + alpha*resoloss(outputa[:,2],trutha[:,1])
    return total_loss


def categorical_loss_only(outputa,trutha):
    total_loss =  losscat(outputa,trutha) 
    return total_loss


def bcategorical_loss(outputa,trutha):
    total_loss = F.binary_cross_entropy(outputa, trutha.float())
    return torch.mean(total_loss)

#model.train()
def train(epoch):
    model.train()
    #torch.cuda.empty_cache()
    scheduler.step()
    loss = []
    for data in tqdm(trainloader):
            data = data.to(device)        
            optimizer.zero_grad()
            #print(data.x.size())
            #print("data.x",data.x.shape)
            #print("data.nodex",data.nodex.shape)
            result = model(data)
            #print("result",result)
            #print("data.edgelabel",data.edgelabel)
            #print(result.shape,data.edgelabel.shape)
            #lossc = focalloss(result, data.edgelabel.type(torch.int64))
            lossc = categorical_loss_only(result, data.edgelabel.type(torch.int64))
            #lossc = bcategorical_loss(result, data.edgelabel.type(torch.int64))
            loss.append(lossc.item()) 
            lossc.backward()
            optimizer.step()
            scheduler.batch_step()
    print( 'batches for train:',len(loss)) 
#    print('loss',loss)
    print('train loss:',np.mean(np.array(loss)))
    return np.mean(np.array(loss))




def gaussian(x,  mean,a, sigma):
    return a * np.exp(-((x - mean)**2 / (2 * sigma**2)))

def evaluate(epoch):
        """"Evaluate the model"""
        model.zero_grad()
        #torch.cuda.empty_cache()
        model.eval()
        pred = []
        true = []
        loss= []
        
        correct = 0
        predc = []
        truec = []
        for data in tqdm(testloader):
            data = data.to(device)        
            result = model(data)
            #lossc = resoloss(result, data.y)
            #lossc = categorical_loss_only(result, data.label)
            #lossc = focalloss(result, data.edgelabel.type(torch.int64))
            lossc = categorical_loss_only(result, data.edgelabel.type(torch.int64))
            #lossc = bcategorical_loss(result, data.edgelabel.type(torch.int64))
#            print (result.item(),data.y.item())
#            frac.append((result.item() - data.y.item())/data.y.item())
            loss.append(lossc.item())

            for i in result:
                #pred.append(i.detach().cpu()[2])
                #predc.append(i.detach().cpu())
                predc.append(i.detach().cpu().argmax())
            for i in data.edgelabel.detach():
                #true.append(i.detach().cpu()[1])
                truec.append(i.detach().cpu().flatten())
                #print(i.detach().cpu())
            
        #print(predc,truec)
        predc = np.array(predc)
        #predc[predc < 0.5] = 0.
        #predc[predc > 0.5] = 1.
        truec = np.array(truec)
        
        #eleacc = [] 
        #puacc = []
        
        totacc = np.equal(predc,truec).sum()/len(truec)
        eleacc = np.equal(predc[truec==1],truec[truec==1]).sum()/len(truec[truec==1])
        puacc = np.equal(predc[truec==0],truec[truec==0]).sum()/len(truec[truec==0])


        return np.mean(np.array(loss)), totacc, eleacc, puacc
        


In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)


from tqdm import tqdm as tqdm
checkpoint_dir = '/home/llr/cms/sghosh/hackathonTICL/model_clusnetwork/'

os.makedirs(checkpoint_dir, exist_ok=True)
best_loss = 99999999
losst = []
lossv = []
epochs = []
acc = []
elc= []
puc = []


for epoch in range(1, 201):
    print ('epoch:',epoch)
    losst.append(train(epoch))
    loss_epoch,accuracy, eleacc, puacc = evaluate(epoch)
    lossv.append(loss_epoch)
    acc.append(accuracy)
    elc.append(eleacc)
    puc.append(puacc)
    epochs.append(epoch)
    print("test total acc, ele acc, pu acc:",accuracy,eleacc,puacc)
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }
    
    
    
    checkpoint_file = 'model_epoch_%i.pth.tar' % ( epoch )
    torch.save(checkpoint,
                   os.path.join(checkpoint_dir,checkpoint_file ))
    if loss_epoch < best_loss:
        best_loss = loss_epoch
        print('new best test loss:',best_loss)
        torch.save(checkpoint,
                   os.path.join(checkpoint_dir,'model_checkpoint_best.pth.tar' ))
    if (epoch%10 == 0):
        plt.plot(np.array(epochs),np.array(losst),c='b',label='training')
        plt.plot(np.array(epochs),np.array(lossv),c='r',label='testing')
        plt.xlabel("epoch")
        plt.ylabel("loss")
        plt.legend()
        plt.savefig(checkpoint_dir+'loss_{0}.png'.format(epoch))
        plt.show()
        plt.plot(np.array(epochs),np.array(acc),c='b')
        plt.xlabel("epoch")
        plt.ylabel("accuracy")
        plt.savefig(checkpoint_dir+'acc_class_{0}.png'.format(epoch))
        plt.show()
        fig, (ax0,ax1) = plt.subplots(1, 2, figsize=(20,12))
        ax0.plot(np.array(epochs),np.array(elc))#,label=str(etabinsG[i])+"_to_"+str(etabinsG[i+1]))
        ax1.set_xlabel("epoch")
        ax1.set_ylabel("accuracy ele")
        ax1.plot(np.array(epochs),np.array(puc))#,label=str(etabinsG[i])+"_to_"+str(etabinsG[i+1]))
        ax1.set_xlabel("epoch")
        ax1.set_ylabel("accuracy pu")
        plt.legend()
        plt.savefig(checkpoint_dir+'acc_class_feta_{0}.png'.format(epoch))
        plt.show()   
            
        
        
